In [4]:
import os
import requests
from bs4 import BeautifulSoup
import matplotlib.pylab as plt
# import utils
import warnings 
warnings.filterwarnings("ignore")
import pandas as pd 


In [5]:
import folium

In [2]:
grid_plot = pd.read_csv("grid_plot.csv")

# Q1a 

In [139]:
url = 'https://www.glerl.noaa.gov/emf/glcfs/gridded_fields/NCAST/'
response = requests.get(url)


In [140]:
soup = BeautifulSoup(response.text, 'html.parser')

In [141]:
table_rows = soup.findAll("tr")

In [142]:
table_rows = table_rows[3:-1]

In [143]:
for table_row in table_rows:
    print(table_row)

<tr><td valign="top"><img alt="[   ]" src="/icons/unknown.gif"/></td><td><a href="bd201835712.1.at">bd201835712.1.at</a></td><td align="right">2018-12-23 13:36  </td><td align="right">239 </td><td> </td></tr>
<tr><td valign="top"><img alt="[TXT]" src="/icons/text.gif"/></td><td><a href="bd201835712.1.cc">bd201835712.1.cc</a></td><td align="right">2018-12-23 13:36  </td><td align="right">239 </td><td> </td></tr>
<tr><td valign="top"><img alt="[   ]" src="/icons/unknown.gif"/></td><td><a href="bd201835712.1.dp">bd201835712.1.dp</a></td><td align="right">2018-12-23 13:36  </td><td align="right">239 </td><td> </td></tr>
<tr><td valign="top"><img alt="[   ]" src="/icons/unknown.gif"/></td><td><a href="bd201835712.1.vac">bd201835712.1.vac</a></td><td align="right">2018-12-23 13:36  </td><td align="right">263 </td><td> </td></tr>
<tr><td valign="top"><img alt="[   ]" src="/icons/unknown.gif"/></td><td><a href="bd201835712.1.wnd">bd201835712.1.wnd</a></td><td align="right">2018-12-23 13:36  </

<tr><td valign="top"><img alt="[   ]" src="/icons/unknown.gif"/></td><td><a href="s202103118.0.cur">s202103118.0.cur</a></td><td align="right">2021-01-31 19:24  </td><td align="right"> 95K</td><td> </td></tr>
<tr><td valign="top"><img alt="[   ]" src="/icons/unknown.gif"/></td><td><a href="s202103118.0.eg">s202103118.0.eg</a></td><td align="right">2021-01-31 19:24  </td><td align="right"> 71K</td><td> </td></tr>
<tr><td valign="top"><img alt="[   ]" src="/icons/unknown.gif"/></td><td><a href="s202103118.0.ice">s202103118.0.ice</a></td><td align="right">2021-01-31 19:24  </td><td align="right">180K</td><td> </td></tr>
<tr><td valign="top"><img alt="[   ]" src="/icons/unknown.gif"/></td><td><a href="s202103118.0.swt">s202103118.0.swt</a></td><td align="right">2021-01-31 19:24  </td><td align="right"> 71K</td><td> </td></tr>
<tr><td valign="top"><img alt="[SND]" src="/icons/sound2.gif"/></td><td><a href="s202103118.0.wav">s202103118.0.wav</a></td><td align="right">2021-01-31 19:24  </td><

# Q1b


In [144]:
tmp_list = []
for table_row in table_rows:
    
    table_row_list = table_row.findAll("td")
    filename = table_row_list[1].find("a").get("href")
    upload_date = table_row_list[2].text.strip()
    file_size = table_row_list[3].text.strip()
    tmp_list.append((filename, upload_date, file_size))


In [145]:
noaa_files = pd.DataFrame(tmp_list, columns=["filename", "upload_date", "file_size"])

In [146]:
noaa_files.head()

,filename,upload_date,file_size
0,bd201835712.1.at,2018-12-23 13:36,239
1,bd201835712.1.cc,2018-12-23 13:36,239
2,bd201835712.1.dp,2018-12-23 13:36,239
3,bd201835712.1.vac,2018-12-23 13:36,263
4,bd201835712.1.wnd,2018-12-23 13:36,263


In [147]:
print("noaa_files has {} rows".format(len(noaa_files)))

noaa_files has 1429 rows


# Q1c


In [148]:
index_records = []
for item in range(len(noaa_files)):
    filename = noaa_files.iloc[item, 0]
    if "o" in filename and "wav" in filename:
        index_records.append(item)
        

In [149]:
noaa_files = noaa_files.loc[index_records]

In [150]:
noaa_files.reset_index(inplace=True)

In [151]:
noaa_files.head(10)

,index,filename,upload_date,file_size
0,1049,o202102506.0.wav,2021-01-25 07:24,110K
1,1055,o202102512.0.wav,2021-01-25 13:24,110K
2,1061,o202102518.0.wav,2021-01-25 19:23,110K
3,1067,o202102600.0.wav,2021-01-26 01:23,110K
4,1073,o202102606.0.wav,2021-01-26 07:23,110K
5,1079,o202102612.0.wav,2021-01-26 13:23,110K
6,1085,o202102618.0.wav,2021-01-26 19:23,110K
7,1091,o202102700.0.wav,2021-01-27 01:23,110K
8,1097,o202102706.0.wav,2021-01-27 07:23,110K
9,1103,o202102712.0.wav,2021-01-27 13:23,110K


# Q2a

In [187]:
def noaa_file_parser(filename):
    file_content = open(filename, "r").read().split("\n")
    # print(file_content)
    year = int(file_content[0].split(" ")[0])
    day = int(file_content[0].split(" ")[1])
    hour = int(file_content[0].split(" ")[2])
    
    l = []
    for item in file_content[1:]:
        tmp_l = item.split(" ")
        data_l = [i for i in tmp_l if i]
        if len(data_l) != 4:
            continue
        grid_number = data_l[0]
        wave_height = data_l[1]
        wave_direction = data_l[2]
        wave_period = data_l[3]
        l.append((filename, year, day, hour, grid_number, wave_height, wave_direction, wave_period))
    
    df = pd.DataFrame(l, columns=["filename", "year", "day", "hour", "grid_number", "wave_height", "wave_direction", "wave_period"])
    return df 

file_grid_data = noaa_file_parser(filename='o202102506.0.wav')
file_grid_data.head(10)

,filename,year,day,hour,grid_number,wave_height,wave_direction,wave_period
0,o202102506.0.wav,2021,25,7,1,0.319,291,2.3
1,o202102506.0.wav,2021,25,7,2,0.301,295,2.2
2,o202102506.0.wav,2021,25,7,3,0.266,297,2.1
3,o202102506.0.wav,2021,25,7,4,0.202,298,1.8
4,o202102506.0.wav,2021,25,7,5,0.307,291,2.3
5,o202102506.0.wav,2021,25,7,6,0.323,294,2.4
6,o202102506.0.wav,2021,25,7,7,0.342,297,2.4
7,o202102506.0.wav,2021,25,7,8,0.369,300,2.5
8,o202102506.0.wav,2021,25,7,9,0.344,305,2.4
9,o202102506.0.wav,2021,25,7,10,0.321,306,2.3


In [192]:
print("file_grid_data has {} rows".format(file_grid_data.shape[0]))

file_grid_data has 4476 rows
